# Creating marekting assets with Gemini 2.0

> adapted from [this notebook example](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/marketing/creating_marketing_assets_gemini_2_0.ipynb)

### imports

In [ ]:
import json
import os

from IPython.display import Markdown, display
from google import genai
from google.genai import types
from google.genai.types import GenerateContentConfig, GoogleSearch, Tool
from pydantic import BaseModel

### env setup

In [ ]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables

PROJECT_ID = "hybrid-vertex"
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_LOCATION", "us-central1")

# Instantiate client for Vertex AI
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

In [3]:
MODEL_ID = "gemini-2.0-flash-001"

## Creating a marketing campaign brief using a past campaign as reference

Let's have a look at a sample past campaign brief

In [4]:
# Set the Cloud Storage path
marketing_brief_file_path = "github-repo/generative-ai/gemini2/use-cases/marketing_example/sample_marketing_campaign_brief.pdf"
marketing_brief_file_uri = f"gs://{marketing_brief_file_path}"
marketing_brief_file_url = f"https://storage.googleapis.com/{marketing_brief_file_path}"

print("Click to view the sample file:")
print(marketing_brief_file_url)

Click to view the sample file:
https://storage.googleapis.com/github-repo/generative-ai/gemini2/use-cases/marketing_example/sample_marketing_campaign_brief.pdf


### Controlled Generation

In [5]:
class MarketingCampaignBrief(BaseModel):
    campaign_name: str
    campaign_objectives: list[str]
    target_audience: str
    media_strategy: list[str]
    timeline: str
    target_countries: list[str]
    performance_metrics: list[str]

### Extract details from sample past campaign brief

1. Send the prompt together with the sample past campaign brief PDF to Gemini 2.0 Flash
2. Specify that Gemini returns the response in the MarketingCampaignBrief schema you defined previously by including `response_schema=MarketingCampaignBrief` in the request

In [6]:
prompt = """
  Extract the details from the sample marketing brief.
"""

marketing_brief_file = types.Part.from_uri(
    file_uri=marketing_brief_file_url, mime_type="application/pdf"
)
contents = [marketing_brief_file, prompt]

response = client.models.generate_content(
    model=MODEL_ID,
    contents=contents,
    config=GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=MarketingCampaignBrief,
    ),
)

sample_marketing_brief = response.text
sample_marketing_brief_json = json.loads(sample_marketing_brief)
print(json.dumps(sample_marketing_brief_json, indent=2))

{
  "campaign_name": "Connect Beyond Limits with Pix Phone 5",
  "campaign_objectives": [
    "Increase awareness of the latest model of the Pix Phone",
    "Generate leads and drive phone sales",
    "Position Pix Phone and the trendy phone to have"
  ],
  "target_audience": "Individuals aged 20-40 in major markets such as US, France, Japan",
  "media_strategy": [
    "Social Media Marketing: Run targeted social media ads on platforms where the target audience is active.",
    "Influencer Marketing: Partner with influencers in the tech industry to promote Pix Phone 5",
    "Paid Advertising: Run targeted display ads on websites and apps frequented by the target audience. Use search engine marketing (SEM) to bid on relevant keywords and appear in search results when potential customers are looking for asset protection insurance."
  ],
  "timeline": "Activity in the 3 major markets in at least the online channels by early Oct 2023, Start from US, followed by France then Japan. The campa

### Google Search as a tool

#### helper functions: grounding with search

In [7]:
def print_grounding_response(response):
    """Prints Gemini response with grounding citations."""
    grounding_metadata = response.candidates[0].grounding_metadata

    # Citation indices are in byte units
    ENCODING = "utf-8"
    text_bytes = response.text.encode(ENCODING)

    prev_index = 0
    markdown_text = ""

    for grounding_support in grounding_metadata.grounding_supports:
        text_segment = text_bytes[
            prev_index : grounding_support.segment.end_index
        ].decode(ENCODING)

        footnotes_text = ""
        for grounding_chunk_index in grounding_support.grounding_chunk_indices:
            footnotes_text += f"[[{grounding_chunk_index + 1}]]({grounding_metadata.grounding_chunks[grounding_chunk_index].web.uri})\n"

        markdown_text += f"{text_segment} {footnotes_text}\n"
        prev_index = grounding_support.segment.end_index

    if prev_index < len(text_bytes):
        markdown_text += str(text_bytes[prev_index:], encoding=ENCODING)

    markdown_text += "\n----\n## Grounding Sources\n"

    if grounding_metadata.web_search_queries:
        markdown_text += (
            f"\n**Web Search Queries:** {grounding_metadata.web_search_queries}\n"
        )
        if grounding_metadata.search_entry_point:
            markdown_text += f"\n**Search Entry Point:**\n {grounding_metadata.search_entry_point.rendered_content}\n"
    elif grounding_metadata.retrieval_queries:
        markdown_text += (
            f"\n**Retrieval Queries:** {grounding_metadata.retrieval_queries}\n"
        )

    markdown_text += "### Grounding Chunks\n"

    for index, grounding_chunk in enumerate(
        grounding_metadata.grounding_chunks, start=1
    ):
        context = grounding_chunk.web or grounding_chunk.retrieved_context
        if not context:
            print(f"Skipping Grounding Chunk {grounding_chunk}")
            continue

        markdown_text += f"{index}. [{context.title}]({context.uri})\n"

    display(Markdown(markdown_text))

#### Grounding + Google Search for market research

In [ ]:

market_research_prompt = """
  I am planning to launch a mobile phone campaign and I want to understand the latest trends in the phone industry.
  Please answer the following questions:
  - What are the latest phone models and their selling point from the top 2 phone makers?
  - What is the general public sentiment about mobile phones?
"""

contents = [market_research_prompt]

google_search_tool = Tool(google_search=GoogleSearch())

response = client.models.generate_content(
    model=MODEL_ID,
    contents=contents,
    config=GenerateContentConfig(tools=[google_search_tool]),
)

market_research = response.text
print_grounding_response(response)

Here's a breakdown of the latest trends in the mobile phone industry, focusing on top manufacturers and public sentiment:

**Latest Phone Models and Selling Points (Top 2 Makers)**

Based on the search results, the top two mobile phone brands are generally considered to be **Apple** and **Samsung.**

*   **Apple:**
    *   **iPhone 16 Pro Max:** It is considered the best iPhone overall, known for its well-rounded user experience, luxurious design, premium services, and versatile camera. The iPhone 16 Pro and Pro Max models have identical cameras, offering a 5x optical zoom and more fine-tuning capabilities than previous iPhones. [[1]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrySkuVVX1lXKz1SzIIsC5ue9oi8pDeuL6IsdHyPH-E13XLAPrVWIPT8Q6O7yRNf5tpHoIPbfkV9nhpeLv-6Z3oABeEP2FXDMEVr8kVnq3AynILclBR2n5bgiEyOSturM3KT3Ym6)


    *   Apple is increasing its use of recycled materials, aiming for 100% recycled cobalt in batteries. [[2]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrzllpx5CE4_gemVOmM-5_ObikLvnn6Duz50CMgoIn4pC85BEH21RgfhbmV8ogt6yDUUozt6dQtX6Wx6iXMnBLrSGeN3B_dGzoc8qqabkB07rPZ_ODHopk5aYF8q4wuXwxc0g0jE_BR93xJVoSwNcJZVvW-v3T1Y9j7glf3tWgn6KMlE3ci2wKeLgxpGUvOeVo6Qqtc4V6mUGzLaPVkk4awBX7keg9wkyw==)



*   **Samsung:**
    *   **Samsung Galaxy S25 Ultra:** As the best premium Android phone, it's like having a full camera bag in your pocket. While not a massive upgrade from the S24 Ultra, it boasts added megapixels and improved performance with the Snapdragon 8 Elite. [[1]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrySkuVVX1lXKz1SzIIsC5ue9oi8pDeuL6IsdHyPH-E13XLAPrVWIPT8Q6O7yRNf5tpHoIPbfkV9nhpeLv-6Z3oABeEP2FXDMEVr8kVnq3AynILclBR2n5bgiEyOSturM3KT3Ym6)

 It's considered Samsung's best camera phone and one of the best overall. [[1]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrySkuVVX1lXKz1SzIIsC5ue9oi8pDeuL6IsdHyPH-E13XLAPrVWIPT8Q6O7yRNf5tpHoIPbfkV9nhpeLv-6Z3oABeEP2FXDMEVr8kVnq3AynILclBR2n5bgiEyOSturM3KT3Ym6)


    *   **Galaxy A Series (e.g., A15, A35, A55):** These entry-level and mid-range phones prioritize affordability. They have received limited AI features via software updates. [[3]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrwQ8B4UBW7lRPfgu5rjc6ttTWhWuzgTacI1JoQvcZOZOnbDrQGD9CJJitg25P2EMoFwzVnaIcdIG3viAzmvAy87OI_wWhGb0tKIuNJ-Ef-OmARet_Mcs2NS8OXZpI9Dcn4fGVCG5f3sir0qk1eLnUwBNU20jRVIg0xCJugb1HCg3kFzjS91fPI3oxZo-yhsKRYn9x18)



**Key Trends and Features to Watch:**

*   **AI Integration:** Artificial Intelligence (AI) is becoming more deeply integrated into mobile phones, personalizing user experiences, optimizing battery life, and automating tasks. [[4]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrwQCwugPsMpioZcwTmCNqtkqlzxiGIWX4ysGgDtzgAW5GEZLOmGoIANMWhGOxcn9zanpCcg7qYT5Xpu37J0vAaTHsNi-I4PzkM7LWx30E5l8lPbBrMC5UH3TzIxRzUcDMVxYfbdI5siCicbFWi6KnWrvcqg42529RHJnY_15672n9tN6oxOienmLhkdgblm0hsKPgbHKlPbPN96_judag==)

 Expect AI-powered cameras that recognize objects, scenes, and emotions for better photos. [[4]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrwQCwugPsMpioZcwTmCNqtkqlzxiGIWX4ysGgDtzgAW5GEZLOmGoIANMWhGOxcn9zanpCcg7qYT5Xpu37J0vAaTHsNi-I4PzkM7LWx30E5l8lPbBrMC5UH3TzIxRzUcDMVxYfbdI5siCicbFWi6KnWrvcqg42529RHJnY_15672n9tN6oxOienmLhkdgblm0hsKPgbHKlPbPN96_judag==)


*   **Foldable and Rollable Screens:** These designs are expected to become more mainstream and refined, offering larger screens for multitasking. [[5]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrzzI5O1UV7kBInBRdb9ITC5Xaeyq5ZA1Lhs1lGa77Z2u_MIUWXYSPSQ56awREzxK1HMYmueGbEpJOCjv_7us3FMWEq6KbnP8DDTM62ONmx4LKBx1qyzvTydhc4-pz7Hq0wPO-MKjr-h3y3HE-k4g1qYqS7izblmxN-JaVoOmb24lYia3Nxaw1QIuRGGSM_KqRuMeGIrEdfVfXhLz8kaYZhH3cCkDhyH)


*   **5G and 6G Transition:** While 5G is becoming dominant, the transition to 6G technology could begin. [[4]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrwQCwugPsMpioZcwTmCNqtkqlzxiGIWX4ysGgDtzgAW5GEZLOmGoIANMWhGOxcn9zanpCcg7qYT5Xpu37J0vAaTHsNi-I4PzkM7LWx30E5l8lPbBrMC5UH3TzIxRzUcDMVxYfbdI5siCicbFWi6KnWrvcqg42529RHJnY_15672n9tN6oxOienmLhkdgblm0hsKPgbHKlPbPN96_judag==)


*   **Extended Reality (XR) and Metaverse Integration:** Augmented reality (AR), virtual reality (VR), and mixed reality (MR) will be more integrated, offering immersive experiences for various applications. [[4]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrwQCwugPsMpioZcwTmCNqtkqlzxiGIWX4ysGgDtzgAW5GEZLOmGoIANMWhGOxcn9zanpCcg7qYT5Xpu37J0vAaTHsNi-I4PzkM7LWx30E5l8lPbBrMC5UH3TzIxRzUcDMVxYfbdI5siCicbFWi6KnWrvcqg42529RHJnY_15672n9tN6oxOienmLhkdgblm0hsKPgbHKlPbPN96_judag==)


*   **Sustainability:** There's an increasing focus on sustainable design and materials in smartphone manufacturing.
*   **Enhanced Security:** Features like theft detection are emerging, leveraging encryption, biometrics, and on-device AI to protect data. [[2]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrzllpx5CE4_gemVOmM-5_ObikLvnn6Duz50CMgoIn4pC85BEH21RgfhbmV8ogt6yDUUozt6dQtX6Wx6iXMnBLrSGeN3B_dGzoc8qqabkB07rPZ_ODHopk5aYF8q4wuXwxc0g0jE_BR93xJVoSwNcJZVvW-v3T1Y9j7glf3tWgn6KMlE3ci2wKeLgxpGUvOeVo6Qqtc4V6mUGzLaPVkk4awBX7keg9wkyw==)


*   **Better Battery Technology:** Improvements in battery technology are expected to enhance battery life.

**General Public Sentiment about Mobile Phones** [[6]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrxwRlI0HwwgVOp0JLmq5tCfzUhTm1706pbmRU8akfMqwnNzo-dtDQZHoo_n5sDW7Y1tmOnmRq3ZzGQRBi-DCsgY0yij15ZQ5IoUf0iuWSMDg8_oYLn95bVc719nhof1xkuiw7GHd5YYfiBQ0HKFvuyqi6NsXA==)
[[7]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrwbpqAHQA_7MWbcOataQSqiwKZsfSzvc9P9Ol017L0avBMdMIT41uLxelGdx4O72teAJzdIv3FOf42SAmTqaA81eqZR0zpnTcAKDWtk95JZYn0fvGdOHFond5mxQ0LxU2eisY5GiMsPHOCMVHPl-s-k3JGds-qknW0FOPs2d3AiJoM=)



*   **Addiction and Overuse:** There's a growing concern about smartphone addiction and the amount of time people spend on their phones. A significant percentage of people feel uncomfortable without their phones and even sleep with them. [[7]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrwbpqAHQA_7MWbcOataQSqiwKZsfSzvc9P9Ol017L0avBMdMIT41uLxelGdx4O72teAJzdIv3FOf42SAmTqaA81eqZR0zpnTcAKDWtk95JZYn0fvGdOHFond5mxQ0LxU2eisY5GiMsPHOCMVHPl-s-k3JGds-qknW0FOPs2d3AiJoM=)


*   **Negative Impact on Mental Health and Attention:** Many school leaders and policymakers believe cell phones negatively impact students' mental health, attention spans, and academic performance. [[6]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrxwRlI0HwwgVOp0JLmq5tCfzUhTm1706pbmRU8akfMqwnNzo-dtDQZHoo_n5sDW7Y1tmOnmRq3ZzGQRBi-DCsgY0yij15ZQ5IoUf0iuWSMDg8_oYLn95bVc719nhof1xkuiw7GHd5YYfiBQ0HKFvuyqi6NsXA==)


*   **Desire for Security:** As phones become more essential, the importance of keeping them secure is increasing.
*   **Positive View of AI:** AI is generally viewed as a key driver of innovation in the smartphone market.
*   **Mobile payments will be easier as well** [[8]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrzv2nedm7012Mzl5yUVYM_vUINrpQW2MS0zMzzngtLZ2FFMLHPXd_-02jDUOdrxX9n4qbIjy7VIYnUCbbb5OPcpnykJf51EbIZZ-4q_gnVPJMo5F-pWDrvHvXZkqsggruN8bIno75w6wZxOeIVDCcMHcbmqU_se1F4_WiY=)


*   **High ownership:** Over 98% of Americans owned a cellphone in 2024 [[7]](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrwbpqAHQA_7MWbcOataQSqiwKZsfSzvc9P9Ol017L0avBMdMIT41uLxelGdx4O72teAJzdIv3FOf42SAmTqaA81eqZR0zpnTcAKDWtk95JZYn0fvGdOHFond5mxQ0LxU2eisY5GiMsPHOCMVHPl-s-k3JGds-qknW0FOPs2d3AiJoM=)



----
## Grounding Sources

**Web Search Queries:** ['top 2 mobile phone makers 2024 2025 latest models selling points', 'latest mobile phone trends 2024 2025', 'public sentiment mobile phones 2024 2025', 'most popular mobile phones 2024 2025']

**Search Entry Point:**
 <style>
.container {
  align-items: center;
  border-radius: 8px;
  display: flex;
  font-family: Google Sans, Roboto, sans-serif;
  font-size: 14px;
  line-height: 20px;
  padding: 8px 12px;
}
.chip {
  display: inline-block;
  border: solid 1px;
  border-radius: 16px;
  min-width: 14px;
  padding: 5px 16px;
  text-align: center;
  user-select: none;
  margin: 0 8px;
  -webkit-tap-highlight-color: transparent;
}
.carousel {
  overflow: auto;
  scrollbar-width: none;
  white-space: nowrap;
  margin-right: -12px;
}
.headline {
  display: flex;
  margin-right: 4px;
}
.gradient-container {
  position: relative;
}
.gradient {
  position: absolute;
  transform: translate(3px, -9px);
  height: 36px;
  width: 9px;
}
@media (prefers-color-scheme: light) {
  .container {
    background-color: #fafafa;
    box-shadow: 0 0 0 1px #0000000f;
  }
  .headline-label {
    color: #1f1f1f;
  }
  .chip {
    background-color: #ffffff;
    border-color: #d2d2d2;
    color: #5e5e5e;
    text-decoration: none;
  }
  .chip:hover {
    background-color: #f2f2f2;
  }
  .chip:focus {
    background-color: #f2f2f2;
  }
  .chip:active {
    background-color: #d8d8d8;
    border-color: #b6b6b6;
  }
  .logo-dark {
    display: none;
  }
  .gradient {
    background: linear-gradient(90deg, #fafafa 15%, #fafafa00 100%);
  }
}
@media (prefers-color-scheme: dark) {
  .container {
    background-color: #1f1f1f;
    box-shadow: 0 0 0 1px #ffffff26;
  }
  .headline-label {
    color: #fff;
  }
  .chip {
    background-color: #2c2c2c;
    border-color: #3c4043;
    color: #fff;
    text-decoration: none;
  }
  .chip:hover {
    background-color: #353536;
  }
  .chip:focus {
    background-color: #353536;
  }
  .chip:active {
    background-color: #464849;
    border-color: #53575b;
  }
  .logo-light {
    display: none;
  }
  .gradient {
    background: linear-gradient(90deg, #1f1f1f 15%, #1f1f1f00 100%);
  }
}
</style>
<div class="container">
  <div class="headline">
    <svg class="logo-light" width="18" height="18" viewBox="9 9 35 35" fill="none" xmlns="http://www.w3.org/2000/svg">
      <path fill-rule="evenodd" clip-rule="evenodd" d="M42.8622 27.0064C42.8622 25.7839 42.7525 24.6084 42.5487 23.4799H26.3109V30.1568H35.5897C35.1821 32.3041 33.9596 34.1222 32.1258 35.3448V39.6864H37.7213C40.9814 36.677 42.8622 32.2571 42.8622 27.0064V27.0064Z" fill="#4285F4"/>
      <path fill-rule="evenodd" clip-rule="evenodd" d="M26.3109 43.8555C30.9659 43.8555 34.8687 42.3195 37.7213 39.6863L32.1258 35.3447C30.5898 36.3792 28.6306 37.0061 26.3109 37.0061C21.8282 37.0061 18.0195 33.9811 16.6559 29.906H10.9194V34.3573C13.7563 39.9841 19.5712 43.8555 26.3109 43.8555V43.8555Z" fill="#34A853"/>
      <path fill-rule="evenodd" clip-rule="evenodd" d="M16.6559 29.8904C16.3111 28.8559 16.1074 27.7588 16.1074 26.6146C16.1074 25.4704 16.3111 24.3733 16.6559 23.3388V18.8875H10.9194C9.74388 21.2072 9.06992 23.8247 9.06992 26.6146C9.06992 29.4045 9.74388 32.022 10.9194 34.3417L15.3864 30.8621L16.6559 29.8904V29.8904Z" fill="#FBBC05"/>
      <path fill-rule="evenodd" clip-rule="evenodd" d="M26.3109 16.2386C28.85 16.2386 31.107 17.1164 32.9095 18.8091L37.8466 13.8719C34.853 11.082 30.9659 9.3736 26.3109 9.3736C19.5712 9.3736 13.7563 13.245 10.9194 18.8875L16.6559 23.3388C18.0195 19.2636 21.8282 16.2386 26.3109 16.2386V16.2386Z" fill="#EA4335"/>
    </svg>
    <svg class="logo-dark" width="18" height="18" viewBox="0 0 48 48" xmlns="http://www.w3.org/2000/svg">
      <circle cx="24" cy="23" fill="#FFF" r="22"/>
      <path d="M33.76 34.26c2.75-2.56 4.49-6.37 4.49-11.26 0-.89-.08-1.84-.29-3H24.01v5.99h8.03c-.4 2.02-1.5 3.56-3.07 4.56v.75l3.91 2.97h.88z" fill="#4285F4"/>
      <path d="M15.58 25.77A8.845 8.845 0 0 0 24 31.86c1.92 0 3.62-.46 4.97-1.31l4.79 3.71C31.14 36.7 27.65 38 24 38c-5.93 0-11.01-3.4-13.45-8.36l.17-1.01 4.06-2.85h.8z" fill="#34A853"/>
      <path d="M15.59 20.21a8.864 8.864 0 0 0 0 5.58l-5.03 3.86c-.98-2-1.53-4.25-1.53-6.64 0-2.39.55-4.64 1.53-6.64l1-.22 3.81 2.98.22 1.08z" fill="#FBBC05"/>
      <path d="M24 14.14c2.11 0 4.02.75 5.52 1.98l4.36-4.36C31.22 9.43 27.81 8 24 8c-5.93 0-11.01 3.4-13.45 8.36l5.03 3.85A8.86 8.86 0 0 1 24 14.14z" fill="#EA4335"/>
    </svg>
    <div class="gradient-container"><div class="gradient"></div></div>
  </div>
  <div class="carousel">
    <a class="chip" href="https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblryg9RcA5-ox1x-sjB3upGgjbOTqkrAxyuUg5ZqVV0skOGSmf9P9mZ3AGrxMrHAoW2XFFU4ALDUVbqZE3wbDMTch8tptRd5p_W7R4H3eqFsUJVJzXZNOyEzOnUNW-v5IJo08TJquxAZgeFwpqPEBVpGws3BYJL_gwi77r1tFQXwo7HAsonr7gHRcYiHqASt0GM6Uig_GmlxOPB9aZKtCpmQtNlabbpAI83TF9Xwbu9xygtP-KcKIarWdhXeaPJju_YFlgKr0">top 2 mobile phone makers 2024 2025 latest models selling points</a>
    <a class="chip" href="https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrxW0NrN2o4aTh0mFBuwBWE0IinlXq3iduNVO1hYdBuM5qf64PdayX8qz62l9QCFOYeHdw_vpctPaPgyFpczb-hpYEK-ODBl_w7HMxAF34BKiDZZ-i_SffWhwlHtVOnnnrTtqPrVLzTUEPs5Jy07h2vqxbN_fZ0cl_KIaJVeUSxDLkX1jJi9bUircDnuWF46nDPgPCgBuxpUChMaqqbSGzHMHrFptIgdBg==">latest mobile phone trends 2024 2025</a>
    <a class="chip" href="https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblryrMNsa_vnQLcNDjxdMQj-BwynRhB-KABKe1aiQyHNzjsvwWtNSCbrAUXI0uim7Ze8j1OBGkn0PedYiU_ylquRvAa6-udY9Hz_jsbqmybjN0TUtra9kapPB88alRxsAXONGHPYHP_YrZ78CAxKoTft_43H8AulexdI8n2f6jhzFz8HjfS9iHQeulNV6VXKDL__Zk7LZCIicH0rcOHN2uu_vgI1sxteoUEawwUY=">public sentiment mobile phones 2024 2025</a>
    <a class="chip" href="https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblryPslBfCtFkGP227RgseCLD-2UxnOxUhWQ-lHjgCicTA_HPzoWzyKuUPZ8zjiUqqVrvkHuYooJ8QbpCkfffb-fuhs0KHnxPRitF7ZzM-0O0oJ1shYmqw98237Z81RMORy-z-NfWvH7RsnM9WlaB0thoyxHxgOo9DH_iPGdZU5nRG_djQTWjwkEGdMzP-abpD1V_f1fPiVbMIfef4YDI_rOaxU9nROyklQ==">most popular mobile phones 2024 2025</a>
  </div>
</div>

### Grounding Chunks
1. [techradar.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrySkuVVX1lXKz1SzIIsC5ue9oi8pDeuL6IsdHyPH-E13XLAPrVWIPT8Q6O7yRNf5tpHoIPbfkV9nhpeLv-6Z3oABeEP2FXDMEVr8kVnq3AynILclBR2n5bgiEyOSturM3KT3Ym6)
2. [forbes.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrzllpx5CE4_gemVOmM-5_ObikLvnn6Duz50CMgoIn4pC85BEH21RgfhbmV8ogt6yDUUozt6dQtX6Wx6iXMnBLrSGeN3B_dGzoc8qqabkB07rPZ_ODHopk5aYF8q4wuXwxc0g0jE_BR93xJVoSwNcJZVvW-v3T1Y9j7glf3tWgn6KMlE3ci2wKeLgxpGUvOeVo6Qqtc4V6mUGzLaPVkk4awBX7keg9wkyw==)
3. [counterpointresearch.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrwQ8B4UBW7lRPfgu5rjc6ttTWhWuzgTacI1JoQvcZOZOnbDrQGD9CJJitg25P2EMoFwzVnaIcdIG3viAzmvAy87OI_wWhGb0tKIuNJ-Ef-OmARet_Mcs2NS8OXZpI9Dcn4fGVCG5f3sir0qk1eLnUwBNU20jRVIg0xCJugb1HCg3kFzjS91fPI3oxZo-yhsKRYn9x18)
4. [medium.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrwQCwugPsMpioZcwTmCNqtkqlzxiGIWX4ysGgDtzgAW5GEZLOmGoIANMWhGOxcn9zanpCcg7qYT5Xpu37J0vAaTHsNi-I4PzkM7LWx30E5l8lPbBrMC5UH3TzIxRzUcDMVxYfbdI5siCicbFWi6KnWrvcqg42529RHJnY_15672n9tN6oxOienmLhkdgblm0hsKPgbHKlPbPN96_judag==)
5. [diyfixtool.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrzzI5O1UV7kBInBRdb9ITC5Xaeyq5ZA1Lhs1lGa77Z2u_MIUWXYSPSQ56awREzxK1HMYmueGbEpJOCjv_7us3FMWEq6KbnP8DDTM62ONmx4LKBx1qyzvTydhc4-pz7Hq0wPO-MKjr-h3y3HE-k4g1qYqS7izblmxN-JaVoOmb24lYia3Nxaw1QIuRGGSM_KqRuMeGIrEdfVfXhLz8kaYZhH3cCkDhyH)
6. [ed.gov](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrxwRlI0HwwgVOp0JLmq5tCfzUhTm1706pbmRU8akfMqwnNzo-dtDQZHoo_n5sDW7Y1tmOnmRq3ZzGQRBi-DCsgY0yij15ZQ5IoUf0iuWSMDg8_oYLn95bVc719nhof1xkuiw7GHd5YYfiBQ0HKFvuyqi6NsXA==)
7. [consumeraffairs.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrwbpqAHQA_7MWbcOataQSqiwKZsfSzvc9P9Ol017L0avBMdMIT41uLxelGdx4O72teAJzdIv3FOf42SAmTqaA81eqZR0zpnTcAKDWtk95JZYn0fvGdOHFond5mxQ0LxU2eisY5GiMsPHOCMVHPl-s-k3JGds-qknW0FOPs2d3AiJoM=)
8. [keebos.com](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AQXblrzv2nedm7012Mzl5yUVYM_vUINrpQW2MS0zMzzngtLZ2FFMLHPXd_-02jDUOdrxX9n4qbIjy7VIYnUCbbb5OPcpnykJf51EbIZZ-4q_gnVPJMo5F-pWDrvHvXZkqsggruN8bIno75w6wZxOeIVDCcMHcbmqU_se1F4_WiY=)


## create marketing campaign

1. Information about the phone that you're launching
2. Prompt to instruct Gemini to create a marketing campaign brief
3. Extracted information from the sample past campaign brief
4. Market research that was done with Grounding with Google Search
5. MarketingCampaignBrief schema that was defined previously

In [9]:
new_phone_details = """
  Phone Name: Pix Phone 10
  Short description: Pix Phone 10 is the flagship phone with a focus on AI-powered features and a completely redesigned form factor.
  Tech Specs:
    - Camera: 50MP main sensor with 48MP ultrawide lens with autofocus for macro shots
    - Performance: P5 processor for fast performance and AI capabilities
    - Battery: 4700mAh battery for all-day usage
  Key Highlights:
    - Powerful camera system
    - Redesigned software user experience to introduce more fun
    - Compact form factor
  Launch timeline: Jan 2025
  Target countries: US, France and Japan
"""

create_brief_prompt = f"""
Given the following details, create a marketing campaign brief for the new phone launch:

Sample campaign brief:
{sample_marketing_brief}

Market research:
{market_research}

New phone details:
{new_phone_details}
"""

contents = [create_brief_prompt]

response = client.models.generate_content(
    model=MODEL_ID,
    contents=contents,
    config=GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=MarketingCampaignBrief,
    ),
)

creative_brief = response.text
creative_brief_json = json.loads(creative_brief)
print(json.dumps(creative_brief_json, indent=2))

{
  "campaign_name": "Experience the Future with Pix Phone 10",
  "campaign_objectives": [
    "Generate excitement and awareness for the Pix Phone 10 launch.",
    "Drive pre-orders and initial sales of Pix Phone 10.",
    "Position Pix Phone 10 as the leading phone for AI-powered experiences and innovative design."
  ],
  "target_audience": "Tech-savvy individuals aged 22-45 in the US, France, and Japan who value cutting-edge technology, sleek design, and advanced camera capabilities.",
  "media_strategy": [
    "Social Media Marketing: Utilize visually engaging content showcasing the phone's design and AI features on platforms like Instagram, TikTok, and YouTube. Run targeted ad campaigns focused on tech enthusiasts and early adopters.",
    "Influencer Marketing: Partner with tech reviewers and lifestyle influencers to create authentic reviews and demonstrations of the Pix Phone 10's key features.",
    "Online Advertising: Employ targeted display ads and search engine marketing (S

# Creating Assets for the Marketing Campaign

In the following sections, we will be looking at creating:

* Social Media Ad Copy
* Storyboarding for short-form videos

## Social Media Ad Copy

1. Define the JSON response schema for our ad copy
2. Send the prompt and response schema to Gemini 2.0 Flash

In [10]:
# JSON response schema for an ad copy

class AdCopy(BaseModel):
    ad_copy_options: list[str]
    localization_notes: list[str]
    visual_description: list[str]

In [11]:
ad_copy_prompt = f"""
  Given the marketing campaign brief, create an Instagram ad-copy for each target market: {creative_brief_json["target_countries"]}
  Please localize the ad-copy and the visuals to the target markets for better relevancy to the target audience.
  Marketing Campaign Brief:
  {creative_brief}
"""

contents = [ad_copy_prompt]

response = client.models.generate_content(
    model=MODEL_ID,
    contents=contents,
    config=GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=AdCopy,
    ),
)

ad_copy = response.text
ad_copy_json = json.loads(ad_copy)
print(json.dumps(ad_copy_json, indent=2, ensure_ascii=False))

{
  "ad_copy_options": [
    "🇺🇸 US: Experience the future with Pix Phone 10! 🚀 AI-powered brilliance meets stunning design. Pre-order now and be among the first to own the next generation. #PixPhone10 #AIphone #Innovation #Tech",
    "🇫🇷 France: Découvrez le futur avec le Pix Phone 10 ! 🚀 L'intelligence artificielle rencontre un design époustouflant. Précommandez maintenant et soyez parmi les premiers à posséder la prochaine génération. #PixPhone10 #IAphone #Innovation #Tech",
    "🇯🇵 Japan: 未来を体験しよう！Pix Phone 10登場！🚀 AIパワーと美しいデザインの融合。予約受付中。次世代スマホをいち早く手に入れよう。#PixPhone10 #AIスマホ #イノベーション #テック"
  ],
  "localization_notes": [
    "US: Ad copy uses American English and focuses on direct call to action (pre-order).",
    "France: Ad copy is translated into French and maintains a similar tone to the US version, emphasizing the discovery of the future.",
    "Japan: Ad copy is translated into Japanese, highlighting the fusion of AI power and beautiful design, and encouraging early adoption."
 

## Storyboard for short-form videos

> Gemini 2.0 Flash to help us brainstorm a storyboard for a short-form video to accompany the phone launch campaign

In [12]:
short_video_prompt = f"""
  Given the marketing campaign brief, create a storyboard for a YouTube Shorts video for target markets: {creative_brief_json["target_countries"]}.
  Please localize the content to the target markets for better relevancy to the target audience.
  Marketing Campaign Brief:
  {creative_brief}

"""

contents = [short_video_prompt]

response = client.models.generate_content(model=MODEL_ID, contents=contents)

short_video_response = response.text
display(Markdown(short_video_response))

Okay, here's a storyboard outline for a YouTube Shorts video promoting the Pix Phone 10, localized for the US, France, and Japan. The core concept is a fast-paced, visually driven showcase of the phone's AI capabilities and sleek design, adapted for each region's cultural nuances.

**Core Concept:** "Unlock the Future in Your Pocket" - Each version highlights a specific AI feature and ties it into a relatable, everyday scenario. The focus is on speed, ease of use, and how the phone simplifies life.

**Music:** Upbeat, trendy, and globally appealing electronic music with subtle regional variations (e.g., a slightly more pop-infused track for the US, a touch of French house for France, and J-Pop/electronic fusion for Japan).

**Overall Style:** Clean, modern, and visually stunning. Use dynamic camera angles, quick cuts, and seamless transitions to keep viewers engaged.  Emphasize the phone's screen and its UI.

**Storyboard Structure (All Versions):**

*   **Opening Scene (0-3 seconds):** A close-up of someone's hand reaching for the Pix Phone 10. The screen lights up, showcasing a visually appealing lock screen or animation. Text overlay: "Pix Phone 10."
*   **Feature Highlight (3-7 seconds):** Demonstrate a key AI feature in action. This is where the localization comes into play.
*   **Benefit Showcase (7-10 seconds):** Show how that feature solves a problem or makes life easier.
*   **Design Showcase (10-12 seconds):**  Quick cuts showcasing the phone's sleek design, various color options, and key physical features (camera, buttons, etc.).
*   **Call to Action (12-15 seconds):**  Text overlay: "Pre-order Now!" or "Learn More at [Website]" A simple animation of the phone rotating, highlighting the pre-order webpage on the phone's screen.

**Localized Storyboard Details:**

**1. US Version:**

*   **Feature Highlight:** AI-powered real-time translation.
    *   **Scene:**  A tourist in a bustling New York City food truck. They're struggling to understand the menu. They use the Pix Phone 10's camera to instantly translate the menu into English.
    *   **Benefit Showcase:** They order confidently, enjoy delicious food, and smile. Text overlay: "Break down barriers. Connect with anyone, anywhere."
*   **Visual Cues:** Energetic city life, diverse ethnicities, fast-paced editing.
*   **Music:** Upbeat pop-infused electronic track.

**2. French Version:**

*   **Feature Highlight:** AI-powered photo enhancement.
    *   **Scene:** Someone taking a photo of the Eiffel Tower with the Pix Phone 10. The photo is slightly blurry or has poor lighting. The AI automatically enhances the image, making it stunning.
    *   **Benefit Showcase:** They instantly share the beautiful photo on social media and get lots of likes. Text overlay: "Capture the beauty. Every shot, perfect."
*   **Visual Cues:** Focus on artistry, elegance, and sophisticated visuals. Eiffel tower in the background.
*   **Music:** French house-inspired electronic music with a touch of romance.

**3. Japanese Version:**

*   **Feature Highlight:** AI-powered smart assistant for scheduling and organization.
    *   **Scene:** A busy professional in Tokyo juggling multiple tasks on their phone. The Pix Phone 10's AI assistant automatically organizes their schedule, sends reminders, and manages emails.
    *   **Benefit Showcase:** The professional is now calm, collected, and productive. They bow slightly in gratitude to their phone. Text overlay: "Simplify your life. AI that works for you."
*   **Visual Cues:** Clean, minimalist aesthetics, bright colors, and focus on efficiency. Modern Tokyo backdrop.
*   **Music:** Upbeat J-Pop/electronic fusion.

**Common Elements Across All Versions:**

*   **Color Palette:** Modern and sleek, using the brand colors of Pix Phone 10.
*   **Font:** Clean and readable, with subtle variations for each region (e.g., a slightly more rounded font for Japan).
*   **Sound Effects:** Crisp and modern sound effects to emphasize interactions with the phone.
*   **Emphasis on User Experience:** Show the phone's interface as intuitive and easy to use.

**Shooting Notes:**

*   Use professional lighting and high-quality camera equipment.
*   Focus on capturing the phone's screen clearly and showcasing its vibrant display.
*   Use close-ups to emphasize the phone's design details.
*   Ensure all actors are diverse and relatable to the target audience.

**Distribution Notes:**

*   Run targeted YouTube ad campaigns to reach tech-savvy individuals in each country.
*   Optimize video titles, descriptions, and tags for each language.
*   Consider using captions in each language to improve accessibility.

By localizing the content and focusing on relatable scenarios, these YouTube Shorts videos will effectively generate excitement and drive pre-orders for the Pix Phone 10 in the US, France, and Japan.
